In [ ]:
# Import OMERO Python BlitzGateway
import omero
from omero.gateway import BlitzGateway
import ezomero
# Import Numpy
import numpy as np
from dotenv import load_dotenv

# Import Python System Packages
import os
import tempfile
import pandas
import warnings

### Setup connection with OMERO

In [ ]:
load_dotenv(override=True)
conn = BlitzGateway(host=os.environ.get("HOST"), username=os.environ.get("USER_NAME"), passwd=os.environ.get("PASSWORD"), secure=True)
print(conn.connect())
conn.c.enableKeepAlive(60)

### Get info from the dataset

In [ ]:
datatype = "Dataset" # "plate", "dataset", "image"
data_id = 	502
nucl_channel = 0

#validate that data_id matches datatype
if datatype == "Plate":
    plate = conn.getObject("Plate", data_id)
    print('Plate Name: ', plate.getName())
elif datatype == "Dataset":
    dataset = conn.getObject("Dataset", data_id)
    print('Dataset Name: ', dataset.getName())
elif datatype == "Image":
    image = conn.getObject("Image", data_id)
    print('Image Name: ', image.getName())

## Get attachment

In [ ]:
#get attachment id from  
annotations = ezomero.get_file_annotation_ids(conn, datatype,data_id)
print(annotations)
df = ezomero.get_table(conn,annotations[0]) #TODO: find the correct attachment if there are multiple

#show table summary
print(df.head())

### get metadata from OMERO per image


In [ ]:
#list columns from table
print(df.columns)
#list unique values from imageID add to list
image_ids = df['imageID'].unique()
metadata = []
for image_id in image_ids:
    file_ann_ids = ezomero.get_map_annotation_ids(conn, 'Image', int(image_id), ns='openmicroscopy.org/omero/experimental/metadata')
    annotation = ezomero.get_map_annotation(conn, file_ann_ids[0])
    annotation.update({"imageID": image_id}) 
    metadata.append(annotation)
#convert metadata list to from list of dict to a pandas dataframe, match the same keys, but check if maybe some keys are missing in some dicts
metadata_df = pandas.DataFrame(metadata)

#join metadata_df to df
df = df.merge(metadata_df, on='imageID', how='left')
print(df.columns)

### Plotting FUCCI intensities as scatter plot

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def create_intensity_plots(df):
    # Create pivot tables for mean and sum intensities
    pivot_mean = df.pivot_table(
        index=['imageID', 'label', 'Treatment'],
        columns='channel',
        values='mean_intensity'
    ).reset_index()
    
    pivot_sum = df.pivot_table(
        index=['imageID', 'label', 'Treatment'],
        columns='channel',
        values='sum_intensity'
    ).reset_index()
    
    # Rename columns for clarity
    pivot_mean.columns.name = None
    pivot_sum.columns.name = None
    
    pivot_mean = pivot_mean.rename(columns={
        1: 'mean_intensity_ch1',
        2: 'mean_intensity_ch2'
    })
    
    pivot_sum = pivot_sum.rename(columns={
        1: 'sum_intensity_ch1',
        2: 'sum_intensity_ch2'
    })
    
    # Create color palette
    unique_treatments = df['Treatment'].unique()
    palette = dict(zip(unique_treatments, sns.color_palette("hsv", len(unique_treatments))))
    
    # Create figure with 4 subplots
    fig, axes = plt.subplots(2, 2, figsize=(20, 16))
    
    # Plot settings for each subplot
    plot_settings = [
        {'data': pivot_mean, 'x': 'mean_intensity_ch1', 'y': 'mean_intensity_ch2', 'log': False, 'ax': axes[0,0], 'title': 'Mean Intensity (Linear)'},
        {'data': pivot_mean, 'x': 'mean_intensity_ch1', 'y': 'mean_intensity_ch2', 'log': True, 'ax': axes[0,1], 'title': 'Mean Intensity (Log)'},
        {'data': pivot_sum, 'x': 'sum_intensity_ch1', 'y': 'sum_intensity_ch2', 'log': False, 'ax': axes[1,0], 'title': 'Sum Intensity (Linear)'},
        {'data': pivot_sum, 'x': 'sum_intensity_ch1', 'y': 'sum_intensity_ch2', 'log': True, 'ax': axes[1,1], 'title': 'Sum Intensity (Log)'}
    ]
    
    # Create each plot
    for settings in plot_settings:
        ax = settings['ax']
        
        # Create scatter plot
        sns.scatterplot(data=settings['data'], 
                       x=settings['x'], 
                       y=settings['y'], 
                       hue='Treatment', 
                       palette=palette, 
                       alpha=0.6,
                       ax=ax)
        
        # Set scales
        if settings['log']:
            ax.set_xscale('log')
            ax.set_yscale('log')
        
        # Set labels
        ax.set_xlabel('Channel 1 Intensity')
        ax.set_ylabel('Channel 2 Intensity')
        ax.set_title(settings['title'])
        
        # Adjust legend
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    plt.show()

# Call the function with your dataframe
create_intensity_plots(df)

In [ ]:
#plot histogram of mean intensity at the different channels, so 4 plots in total
def create_intensity_histograms(df):
    # Create pivot tables for mean and sum intensities
    pivot_mean = df.pivot_table(
        index=['imageID', 'label', 'Treatment'],
        columns='channel',
        values='mean_intensity'
    ).reset_index()
    
    pivot_sum = df.pivot_table(
        index=['imageID', 'label', 'Treatment'],
        columns='channel',
        values='sum_intensity'
    ).reset_index()
    
    # Rename columns for clarity
    pivot_mean.columns.name = None
    pivot_sum.columns.name = None
    
    pivot_mean = pivot_mean.rename(columns={
        0: 'mean_intensity_ch1',
        1: 'mean_intensity_ch2',
        2: 'mean_intensity_ch3'
    })
    
    pivot_sum = pivot_sum.rename(columns={
        0: 'sum_intensity_ch1',
        1: 'sum_intensity_ch2',
        2: 'sum_intensity_ch3'
    })
    
    # Create color palette
    unique_treatments = df['Treatment'].unique()
    palette = dict(zip(unique_treatments, sns.color_palette("hsv", len(unique_treatments))))
    
    # Create figure with 4 subplots
    fig, axes = plt.subplots(3, 1, figsize=(20, 16))
    # Plot settings for each subplot
    plot_settings = [
        {'data': pivot_mean, 'x': 'mean_intensity_ch1',  'log': False, 'ax': axes[0], 'title': 'Mean Intensity ch1 (Linear)'},
        {'data': pivot_mean, 'x': 'mean_intensity_ch2',  'log': False, 'ax': axes[1], 'title': 'Mean Intensity ch2 (Linear)'},
        {'data': pivot_mean, 'x': 'mean_intensity_ch3',  'log': False, 'ax': axes[2], 'title': 'Mean Intensity ch3 (Linear)'}      
    ]
    
    # Create each plot
    for settings in plot_settings:
        ax = settings['ax']
        
        # Create scatter plot
        sns.histplot(data=settings['data'], 
                       x=settings['x'],
                       ax=ax)
        
        # Set scales
        if settings['log']:
            ax.set_xscale('log')
            ax.set_yscale('log')
        
        # Set labels
        ax.set_xlabel(settings['x'])
        ax.set_ylabel('Frequency')
        ax.set_title(settings['title'])
        
        # Adjust legend
    #    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    plt.show()

# Call the function with your dataframe
create_intensity_histograms(df)

## Plot z- dependent intensity

In [ ]:
df

# plot a histogram of the mean intensity of all nuclei per slice for a single image in the df dataframe, filter also for channel
def plot_nuclei_intensity_histogram(df, image_id, channel):
    # Filter data for the given image and channel
    data = df[(df['imageID'] == image_id) & (df['channel'] == channel)]
    #or for all
    #data = df[(df['channel'] == channel)]
    
    # Create figure and axis
    fig, axes = plt.subplots(1, 2, figsize=(20, 16))
    
    # calculate the number of z-slices (z)
    z = data['z'].nunique()
    print(z)

    #calculate mean intensity of all nuclei per slice
    mean_intensity = data.groupby('z')['mean_intensity'].mean()

    # Create barplot with mean intensity per slice
    sns.barplot(x=mean_intensity.index, y=mean_intensity.values, ax=axes[0])

    # another plot with number of nuclei per slice
    sns.barplot(x=data['z'].value_counts().index, y=data['z'].value_counts().values, ax=axes[1])

    plt.show()

# Call the function with your dataframe

plot_nuclei_intensity_histogram(df, 1511, 0)

In [ ]:
df

### classify Cell Cycle

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def classify_cell_cycle(row, threshold_green, threshold_red):
    """
    Classify cells into cell cycle phases based on FUCCI intensities
    Arguments:
        row: DataFrame row containing mean intensities
        threshold_green: Threshold for green channel (ch1)
        threshold_red: Threshold for red channel (ch2)
    Returns:
        Cell cycle phase classification
    """
    
    green = row['mean_intensity_ch1']  # Green - Geminin (S/G2/M)
    red = row['mean_intensity_ch2']    # Red - CDT1 (G1)
    
    if green < threshold_green and red < threshold_red:
        return 'G0'  # Double negative
    elif green < threshold_green and red > threshold_red:
        return 'G1'  # Red positive only
    elif green > threshold_green and red < threshold_red:
        return 'G2/M'  # Green positive only
    else:
        return 'G1/S'  # Double positive

def plot_cell_cycle_distribution(df, day, threshold_green=500, threshold_red=500, all_wells=False):
    """
    Create stacked bar plot of cell cycle distribution
    Arguments:
        df: DataFrame with intensity measurements
        day: Day of measurement for plot title
        threshold_green: Threshold for green channel classification
        threshold_red: Threshold for red channel classification
    """
    # Pivot and classify cells
    #df = df[df['z']==5]
    pivot = df.pivot_table(
        index=['imageID', 'label', 'Treatment'],
        columns='channel',
        values='mean_intensity'
    ).reset_index()
    
    pivot.columns.name = None
    pivot = pivot.rename(columns={1: 'mean_intensity_ch1', 2: 'mean_intensity_ch2'})
    
    # Classify each cell
    pivot['cell_cycle'] = pivot.apply(lambda row: classify_cell_cycle(row, threshold_green, threshold_red), axis=1)
    
    # Calculate percentages per treatment
    if (all_wells):
        phase_counts = pivot.groupby(['Treatment', 'imageID', 'cell_cycle']).size().unstack(fill_value=0)
    else:
        phase_counts = pivot.groupby(['Treatment', 'cell_cycle']).size().unstack(fill_value=0)

    phase_percentages = phase_counts.div(phase_counts.sum(axis=1), axis=0) * 100
    
    # Plot settings
    colors = {'G0': '#00A0FF', 'G1': '#FF0000', 'G1/S': '#FFFF00', 'G2/M': '#00FF00'}
    
    # Create plot
    ax = phase_percentages.plot(kind='bar', stacked=True, figsize=(15, 6), 
                              color=[colors[x] for x in phase_percentages.columns])
    
    plt.title(f'Cell Cycle Distribution - Day {day}')
    plt.xlabel('Treatment')
    plt.ylabel('% of cells')
    plt.legend(title='Phase', bbox_to_anchor=(1.05, 1))
    plt.tight_layout()
    
    return pivot

# Example usage:
plot_cell_cycle_distribution(df, day=1, threshold_green=100, threshold_red=100,all_wells=False)
plot_cell_cycle_distribution(df, day=1, threshold_green=100, threshold_red=100,all_wells=True)

### Ratio Distribution per treatment

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# First normalize the intensities for each channel
# Method 1: Z-score normalization per channel
pivot_df['norm_ch1'] = (pivot_df['mean_intensity_ch1'] - pivot_df['mean_intensity_ch1'].mean()) / pivot_df['mean_intensity_ch1'].std()
pivot_df['norm_ch2'] = (pivot_df['mean_intensity_ch2'] - pivot_df['mean_intensity_ch2'].mean()) / pivot_df['mean_intensity_ch2'].std()

# Method 2: Min-Max scaling per channel
pivot_df['scaled_ch1'] = (pivot_df['mean_intensity_ch1'] - pivot_df['mean_intensity_ch1'].min()) / (pivot_df['mean_intensity_ch1'].max() - pivot_df['mean_intensity_ch1'].min())
pivot_df['scaled_ch2'] = (pivot_df['mean_intensity_ch2'] - pivot_df['mean_intensity_ch2'].min()) / (pivot_df['mean_intensity_ch2'].max() - pivot_df['mean_intensity_ch2'].min())

# Calculate ratios
pivot_df['ratio_norm'] = pivot_df['norm_ch2'] / pivot_df['norm_ch1']
pivot_df['ratio_scaled'] = pivot_df['scaled_ch2'] / pivot_df['scaled_ch1']

# Create visualization of the ratios
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot density of ratios per treatment
sns.boxplot(data=pivot_df, x='Treatment', y='ratio_norm', ax=ax1)
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right')
ax1.set_title('Ratio Distribution per Treatment (Min-Max scaled)')

# Create 2D density plot of normalized channels
sns.kdeplot(data=pivot_df, x='norm_ch1', y='norm_ch2', hue='Treatment', ax=ax2)
ax2.set_title('2D Density of Normalized Channels')

plt.tight_layout()
plt.show()